# Testing OpenCV algorithms

In [2]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

In [12]:
# Load the pathimage
path = 'Data/retinal-lesions-v20191227/images_896x896'

for idx, imp in enumerate(os.listdir(path)[11:18]):
    im_path = os.path.join(path, imp)
    orig_img = cv2.imread(im_path)
    img = orig_img.copy()

    # Extract the green plane
    green_plane = img[:, :, 1]
    green_orig = green_plane.copy()

    cv2.imshow('Original Image', orig_img)
    cv2.waitKey(0)
    cv2.imshow('Green Plane Image', green_orig)
    cv2.waitKey(0)

    for t in [22,24,27]:
        for c in [4.5, 5]:
            # Define the FA-CLAHE algorithm
            green_plane = green_orig.copy()
            faclahe = cv2.createCLAHE(clipLimit=c, tileGridSize=(t, t))

            # Apply the FA-CLAHE algorithm to the green plane
            faclahe_img = faclahe.apply(green_plane)

            # Replace the green plane with the enhanced green plane
            img[:, :, 1] = faclahe_img

            # Display the original and enhanced images
            cv2.imshow(f'FA-CLAHE Enhanced Image {t}-{c}', faclahe_img)
            cv2.waitKey(0)
        
    cv2.destroyAllWindows()

In [1]:
import cv2
import os
import numpy as np


labels_path = 'Data/retinal-lesions-v20191227/lesion_segs_896x896/labels'
images_path = 'Data/retinal-lesions-v20191227/images_896x896/'

cap2_saving_path = 'cap2_imgs'

if not os.path.exists(cap2_saving_path):
    os.mkdir(cap2_saving_path)

for lb_path in [os.path.join(labels_path, label) for label in os.listdir(labels_path)]:
    # Load the input image in grayscale mode
    lesion_name = lb_path.split('/')[-1].split('__')[-1].split('.')[0]
    image_name = lb_path.split('/')[-1].split('__')[0]
    if lesion_name == 'retinal_hemorrhage':
        mask = cv2.imread(lb_path, cv2.IMREAD_GRAYSCALE)
        sample_image = cv2.imread(os.path.join(images_path,image_name + '.jpg'))
        
        height, width = mask.shape
        # Apply binary thresholding to convert the image to a binary image
        ret, thresh = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Get the connected components of the binary image
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)

        green_plane = sample_image[:, :, 1]

        # Define the FA-CLAHE algorithm
        faclahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(22, 22))

        # Apply the FA-CLAHE algorithm to the green plane
        faclahe_img = faclahe.apply(green_plane.copy())

        # Apply gaussian filtering 
        sigma_space = 4
        filtered = cv2.bilateralFilter(faclahe_img.copy(), 11, sigma_space*3, sigma_space)

        # Resize the image
        resized = cv2.resize(filtered, (640,640), interpolation=cv2.INTER_AREA)
        r_h, r_w = resized.shape

        # Iterate over the connected components and display them one by one
        objects = []
        
        for i in range(1, num_labels):
            # coords = {}

            # coords['x0']= float(stats[i, cv2.CC_STAT_LEFT]) / width
            # coords['y0']= float(stats[i, cv2.CC_STAT_TOP]) / height
            # coords['w']=  float(stats[i, cv2.CC_STAT_WIDTH]) /width
            # coords['h']=  float(stats[i, cv2.CC_STAT_HEIGHT]) / height

            # print(coords)

            # bndbox = np.array(list(coords.values()))
            # bndbox = [str(b) for b in bndbox.tolist()]

            # label = [str(1)]
            # label.extend(bndbox)
            # label = ' '.join(label)

            # print(label)
            # objects.append(label)

            x = stats[i, cv2.CC_STAT_LEFT]
            y = stats[i, cv2.CC_STAT_TOP]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            
            x_r = int(x * (r_w / width))
            y_r = int(y * (r_h / height))
            w_r = int(w * (r_w / width))
            h_r = int(h * (r_h / height))
            (cX, cY) = centroids[i]

            # print(f"[INFO] Component ({i+1}/{num_labels})")
            cv2.rectangle(sample_image, (x, y), (x + w, y + h), (255, 255, 255), 2)
            cv2.rectangle(faclahe_img, (x, y), (x + w, y + h), (255, 255, 255), 2)
            cv2.rectangle(filtered, (x, y), (x + w, y + h), (255, 255, 255), 2)
            cv2.rectangle(resized, (x_r, y_r), (x_r + w_r, y_r + h_r), (255, 255, 255), 2)
            # cv2.circle(sample_image, (int(cX), int(cY)), 4, (0, 0, 255), -1)
            # Create a binary image for the current component
            # component = (labels == i).astype(np.uint8) * 255
            
            # Display the current component
            # cv2.imshow('Component {}'.format(i), component)
        cv2.imshow(f"{lesion_name.title()} in original image", sample_image)
        cv2.imwrite(os.path.join(cap2_saving_path, f'{lesion_name}-original.jpg'), sample_image)
        cv2.waitKey(0)
        cv2.imshow(f"{lesion_name.title()} in green plane of the image", sample_image[:, :, 1])
        cv2.imwrite(os.path.join(cap2_saving_path, f'{lesion_name}-green.jpg'), sample_image[:, :, 1])
        cv2.waitKey(0)
        cv2.imshow(f"{lesion_name.title()} in clahed image", faclahe_img)
        cv2.imwrite(os.path.join(cap2_saving_path, f'{lesion_name}-clahe.jpg'), faclahe_img)
        cv2.waitKey(0)
        cv2.imshow(f"{lesion_name.title()} in filtered image", filtered)
        cv2.imwrite(os.path.join(cap2_saving_path, f'{lesion_name}-filtered.jpg'), filtered)
        cv2.waitKey(0)
        cv2.imshow(f"{lesion_name.title()} in resized image", resized)
        
        # Wait for a key event
        cv2.waitKey(0)

        # Close all open windows
        cv2.destroyAllWindows()


In [3]:
os.path.exists('mask.png')

False